In [1]:
# Loading Packages

import requests
import pandas as pd
import json

#### Elsevier API (Scopus Search)

In [76]:
# API Key and token
elsevier_apiKey= '4fec016525bc7cf8febea7c81db45976'
elsevier_insttoken = '03a9e22c2f85af0fcdd7f9415998446b'

# Define the base URL of the API endpoint
search_url = 'http://api.elsevier.com/content/search/scopus'

# Define your search query
query = 'TITLE-ABS-KEY(GIS)'  # returns documents where the terms appear in the title, keywords, or abstract.


In [87]:
def elsevier_api(search_url, query, start= start_index):
    headers={"Accept":"application/json", "X-ELS-APIKey": elsevier_apiKey,"X-ELS-Insttoken": elsevier_insttoken}
    response = requests.get(f"{search_url}?query={query}&date=2013-2023&view=complete",headers=headers)
    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        try:
            # Attempt to decode the JSON response
            return response.json()
        except json.JSONDecodeError:
            print("Error: Unable to decode JSON response")
            return None, None
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return None, None

In [88]:
# Test elsevier_api function

elsevier_api(search_url, query)

{'search-results': {'opensearch:totalResults': '84826',
  'opensearch:startIndex': '0',
  'opensearch:itemsPerPage': '25',
  'opensearch:Query': {'@role': 'request',
   '@searchTerms': 'TITLE-ABS-KEY(GIS)',
   '@startPage': '0'},
  'link': [{'@_fa': 'true',
    '@ref': 'self',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=TITLE-ABS-KEY%28GIS%29&date=2013-2023&view=complete',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'first',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=TITLE-ABS-KEY%28GIS%29&date=2013-2023&view=complete',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'next',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=25&count=25&query=TITLE-ABS-KEY%28GIS%29&date=2013-2023&view=complete',
    '@type': 'application/json'},
   {'@_fa': 'true',
    '@ref': 'last',
    '@href': 'https://api.elsevier.com/content/search/scopus?start=4975&count=25&que

In [89]:
# Code to extract relevant information from search results

def elsevier_scroll(search_url, query):
    all_results = []
    total_results = None # instead of 0, because the json returns a string
    items_per_page = 25 # this is the maximum number of items per page
    start_index = 0 # First item index. This is always 0.
    
    while total_results is None or start_index <= total_results:
        response = elsevier_api(search_url, query, start=start_index)
        
        if response is None:
            break
        
        search_results = response.get('search-results', {})
        total_results = int(search_results.get('opensearch:totalResults', 0)) # total number of results or 0 if not found
        entries = search_results.get('entry', [])
        
        for entry in entries:
            extracted_info = extract_info(entry)
            all_results.append(extracted_info)
        
        start_index += items_per_page # Pagination
        print(f"Progress: {start_index}/{total_results}") # keeping track of progress
    
    return all_results

def extract_info(entry):
    affiliation_country = entry.get("affiliation", [{}])[0].get("affiliation-country", "")
    return {
        "DOI": entry.get("prism:doi", ""),
        "Title": entry.get("dc:title", ""),
        "First Author": entry.get("dc:creator", ""),
        "Location": affiliation_country,  # Corrected key name
        "Publication Date": entry.get("prism:coverDate", ""),
        "Publication Name": entry.get("prism:publicationName", "")
    }

In [92]:
elsevier_results = elsevier_scroll(search_url, query)

Progress: 25/84826
Progress: 50/84826
Progress: 75/84826
Progress: 100/84826
Progress: 125/84826
Progress: 150/84826
Progress: 175/84826
Progress: 200/84826
Progress: 225/84826
Progress: 250/84826
Progress: 275/84826
Progress: 300/84826
Progress: 325/84826
Progress: 350/84826
Progress: 375/84826
Progress: 400/84826
Progress: 425/84826
Progress: 450/84826
Progress: 475/84826
Progress: 500/84826
Progress: 525/84826
Progress: 550/84826
Progress: 575/84826
Progress: 600/84826
Progress: 625/84826
Progress: 650/84826
Progress: 675/84826
Progress: 700/84826
Progress: 725/84826
Progress: 750/84826
Progress: 775/84826
Progress: 800/84826
Progress: 825/84826
Progress: 850/84826
Progress: 875/84826
Progress: 900/84826
Progress: 925/84826
Progress: 950/84826
Progress: 975/84826
Progress: 1000/84826
Progress: 1025/84826
Progress: 1050/84826
Progress: 1075/84826
Progress: 1100/84826
Progress: 1125/84826
Progress: 1150/84826
Progress: 1175/84826
Progress: 1200/84826
Progress: 1225/84826
Progress: 125

In [93]:
# Converting all_results to a dataframe

elsevier = pd.DataFrame(elsevier_results)
elsevier.head(5)

,DOI,Title,First Author,Location,Publication Date,Publication Name
0,,Assessment of Coastal Water Quality by Testing...,Ezzghari Y.,Morocco,2023-12-31,International Journal of Euro-Mediterranean St...
1,10.1515/9783111317779-004,Menocchio mapped: Italian microhistory and the...,Madden A.,United States,2023-12-31,Zoomland: Exploring Scale in Digital History a...
2,10.21077/ijf.2023.70.4.134234-07,Assessing land suitability for sustainable aqu...,Kumar T.,India,2023-12-31,Indian Journal of Fisheries
3,10.23858/SA/75.2023.2.3374,Attempts at spatial analyses of data from the ...,Solecki R.,Poland,2023-12-31,Sprawozdania Archeologiczne
4,10.21138/GF.825,MEASUREMENT OF SPATIAL INEQUITY IN THE ACCESSI...,Bosisio A.,Spain,2023-12-31,GeoFocus


In [94]:
# Export dataframe as a json file

elsevier.to_json('elsevier_data.json', orient='records')


In [2]:
# Read Elsevier .json file

with open('elsevier_data.json', 'r') as f:
    elsevier_data = json.load(f)

# Converting json to dataframe

elsevier_df = pd.DataFrame(elsevier_data)
elsevier_df.head(5)

,DOI,Title,First Author,Location,Publication Date,Publication Name
0,,Assessment of Coastal Water Quality by Testing...,Ezzghari Y.,Morocco,2023-12-31,International Journal of Euro-Mediterranean St...
1,10.1515/9783111317779-004,Menocchio mapped: Italian microhistory and the...,Madden A.,United States,2023-12-31,Zoomland: Exploring Scale in Digital History a...
2,10.21077/ijf.2023.70.4.134234-07,Assessing land suitability for sustainable aqu...,Kumar T.,India,2023-12-31,Indian Journal of Fisheries
3,10.23858/SA/75.2023.2.3374,Attempts at spatial analyses of data from the ...,Solecki R.,Poland,2023-12-31,Sprawozdania Archeologiczne
4,10.21138/GF.825,MEASUREMENT OF SPATIAL INEQUITY IN THE ACCESSI...,Bosisio A.,Spain,2023-12-31,GeoFocus
